# 파이썬의 데코레이터  

classmethod, staticmethod 같은 함수가 원래 메서드의 정의를 변형하는데 사용되고 있었기 때문에 고안된 수단인데 이 방법은 추가 코드가 필요하고 함수의 원래 정의를 수정해야만 했다.  

데코레이터는 데코레이터 이후에 나오는 것을 데코레이터의 1번째 파라메타로 하고 데코레이터의 결과 값을 반환하게 한다.  
    --> 데코레이터라는 이름은 래핑된 함수의 기능을 수정하고 확장하기 때문에 데코레이터 디자인 패턴과 혼동하면 안 된다.  


## 함수 데코레이터   

함수에 데코레이터를 사용하면 어떤 종류의 로직이라도 적용할 수 있다.  
    -> 파라메타의 유효성을 검사하거나 사전조건을 검사하거나, 기능 전체를 새롭게 정의할 수도 있고, 서명을 변경할 수 있고, 원래 함수의 결과를 캐시하는 등의 작업을 모두 할 수 있다.  

In [ ]:
import logging as logger
from functools import wraps


class ControlledException(Exception):
    '''도메인에서 발생하는 일반적인 예외'''


def retry(operation):
    @wraps(operation)
    def wrapped(*args, **kwargs):
        last_raised = None
        RETRIES_LIMIT = 3
        for _ in range(RETRIES_LIMIT):
            try:
                return operation(*args, **kwargs)
            except ControlledException as e:
                logger.info("retrying %s", operation.__qualname__)
                last_raised = e
    return wrapped

## 클래스 데코레이터  

- 클래스 데코레이터는 코드 재사용과 DRY 원칙의 모든 이점을 공유한다.  
- 당장은 작고 간단한 클래스를 생성하고 나중에 데코레이터로 기능을 보강할 수 있다.  
- 어떤 클래스에 대해서는 유지보수 시 데코레이터를 사용해 기존 로직을 훨씬 쉽게 변경할 수 있다.  



In [ ]:
class LoginEventSerializer:
    def __init__(self, event):
        self.event = event


    def serialize(self) -> dict:
        return {"username": self.event.user_name, "password": '민감한 정보 삭제', "ip": self.event.ip, 
        "timestamp": self.event.timestamp.strftime("%Y-%m-%d %H:%M")}
    

class LoginEvent:
    SERIALIZER = LoginEventSerializer

    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = self.timestamp

    def serialize(self) -> dict:
        return self.SERIALIZER(self).serialize()
